In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [8]:
#Load cleaned dataset
data = pd.read_csv('../../Results/Cleaned_JobDescs.csv', header = 0, names = ['Query', 'Description'])
#data = pd.read_csv('../../Results/Cleaned_JobsNonIT.csv', header = 0, names = ['Query', 'Description'])

In [9]:
#Split the dataset to Training and Test subsets (90/10)
train, test = train_test_split(data, test_size = 0.1, random_state = 17) #random_state = None

train_descs = train['Description']
train_labels = train['Query']
 
test_descs = test['Description']
test_labels = test['Query']

In [12]:
# Model Parameters
num_labels = len(train_labels.unique())
vocab_size = 2000
batch_size = 32
nb_epoch = 50

In [13]:
# Convert Texts to Numeric Vectors for Input
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_descs)
x_train = tokenizer.texts_to_matrix(train_descs, mode = 'tfidf') #count, tfidf, binary
x_test = tokenizer.texts_to_matrix(test_descs, mode = 'tfidf')

#print(tokenizer.word_index[0:5000]) #see the first 5000 corpus words, ordered by frequency

encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [14]:
model = Sequential()
model.add(Dense(2000, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1000))
model.add(Activation('tanh'))
model.add(Dropout(0.1))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'sgd', #'sgd', 'adam', 'RMSprop', 'Adagrad'
              metrics = [metrics.categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
activation_1 (Activation)    (None, 2000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
activation_2 (Activation)    (None, 1000)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               500500    
__________

In [15]:
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = True,
                    validation_split = 0.2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/50
7200/7200 [==============================] - 17s 2ms/step - loss: 2.7062 - categorical_accuracy: 0.2350 - val_loss: 2.0554 - val_categorical_accuracy: 0.4361
Epoch 2/50
7200/7200 [==============================] - 17s 2ms/step - loss: 1.6793 - categorical_accuracy: 0.5258 - val_loss: 1.5881 - val_categorical_accuracy: 0.5394
Epoch 3/50
7200/7200 [==============================] - 17s 2ms/step - loss: 1.1714 - categorical_accuracy: 0.6658 - val_loss: 1.4221 - val_categorical_accuracy: 0.5778
Epoch 4/50
7200/7200 [==============================] - 17s 2ms/step - loss: 0.8366 - categorical_accuracy: 0.7663 - val_loss: 1.3425 - val_categorical_accuracy: 0.5983
Epoch 5/50
7200/7200 [==============================] - 17s 2ms/step - loss: 0.6028 - categorical_accuracy: 0.8408 - val_loss: 1.2831 - val_categorical_accuracy: 0.6139
Epoch 6/50
7200/7200 [==============================] - 17s 2ms/step - loss: 0.4368 - categorical_accuracy:

Epoch 49/50
7200/7200 [==============================] - 23s 3ms/step - loss: 0.0353 - categorical_accuracy: 0.9893 - val_loss: 1.6459 - val_categorical_accuracy: 0.6289
Epoch 50/50
7200/7200 [==============================] - 21s 3ms/step - loss: 0.0353 - categorical_accuracy: 0.9889 - val_loss: 1.6485 - val_categorical_accuracy: 0.6256


In [16]:
score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = True)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])


1000/1000 [==============================] - 0s 385us/step

Test categorical_crossentropy: 1.5726948595046997
Categorical accuracy: 0.625
